In [1]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
#! git clone https://github.com/rapidsai/rapidsai-csp-utils.git
#! python rapidsai-csp-utils/colab/pip-install.py

In [2]:
#! wget https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

In [3]:
#! unzip /content/datos_abiertos_covid19.zip

In [4]:
%load_ext cudf.pandas
# pandas API is now GPU accelerated

import pandas as pd

import cuml
import cuml.preprocessing
import cuml.preprocessing.LabelEncoder

#Quitar warning
import warnings

warnings.filterwarnings("ignore")

In [5]:
import numpy as np


data=pd.read_csv('COVID19MEXICO.csv')
data.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1186694 entries, 0 to 1186693
Data columns (total 40 columns):
 #   Column                 Non-Null Count    Dtype
---  ------                 --------------    -----
 0   FECHA_ACTUALIZACION    1186694 non-null  object
 1   ID_REGISTRO            1186694 non-null  object
 2   ORIGEN                 1186694 non-null  int64
 3   SECTOR                 1186694 non-null  int64
 4   ENTIDAD_UM             1186694 non-null  object
 5   SEXO                   1186694 non-null  int64
 6   ENTIDAD_NAC            1186694 non-null  object
 7   ENTIDAD_RES            1186694 non-null  object
 8   MUNICIPIO_RES          1186694 non-null  object
 9   TIPO_PACIENTE          1186694 non-null  int64
 10  FECHA_INGRESO          1186694 non-null  object
 11  FECHA_SINTOMAS         1186694 non-null  object
 12  FECHA_DEF              1186694 non-null  object
 13  INTUBADO               1186694 non-null  int64
 14  NEUMONIA               1186694 non-null

In [6]:
df=data

df = df.drop(columns=['SECTOR','MUNICIPIO_RES','HABLA_LENGUA_INDIG','ID_REGISTRO','ORIGEN','PAIS_NACIONALIDAD','PAIS_ORIGEN','INDIGENA','ENTIDAD_UM','ENTIDAD_NAC','NACIONALIDAD','ENTIDAD_RES'])
df.columns

# Hacer drop de filas donde "CLASIFICACION_FINAL" sea 2, 4 o 5
df=df.drop(df[df['CLASIFICACION_FINAL']==2].index)
df=df.drop(df[df['CLASIFICACION_FINAL']==4].index)
df=df.drop(df[df['CLASIFICACION_FINAL']==5].index)

# Si alguna fila tiene como valor 97, 98, 99, borrarla
df=df.drop(df[df['FECHA_SINTOMAS']==97].index)
df=df.drop(df[df['FECHA_SINTOMAS']==98].index)
df=df.drop(df[df['FECHA_SINTOMAS']==99].index)


df['FECHA_ACTUALIZACION']=pd.to_datetime(df['FECHA_ACTUALIZACION'])
df['FECHA_INGRESO']=pd.to_datetime(df['FECHA_INGRESO'])
df['FECHA_SINTOMAS']=pd.to_datetime(df['FECHA_SINTOMAS'])
df['FECHA_DEF']=pd.to_datetime(df['FECHA_DEF'],errors='coerce')


for col in ['TIPO_PACIENTE']:
  df[col]=df[col].astype(str)


le = cuml.preprocessing.LabelEncoder()


for col in ['TIPO_PACIENTE','SEXO']:
  df[col]=le.fit_transform(df[col])
# crear caracteristicas apartir de las fechas

df['DIAS_DEFUNCION']=(df['FECHA_ACTUALIZACION']- df['FECHA_DEF']).dt.days
df['DIAS_HOSPITALIZACION']=(df['FECHA_ACTUALIZACION']- df['FECHA_INGRESO']).dt.days
df['DIAS_SINTOMAS']=(df['FECHA_ACTUALIZACION']- df['FECHA_SINTOMAS']).dt.days


# Red NUERONAL REVISAR ESTO
df['DIAS_DEFUNCION'] = df['DIAS_DEFUNCION'].apply(lambda x: 0 if x < 0 else x)



df = df.drop(columns=['FECHA_ACTUALIZACION', 'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF','DIAS_DEFUNCION','MIGRANTE'])

X=df.drop(columns=['CLASIFICACION_FINAL'])
y=df['CLASIFICACION_FINAL']

In [7]:
from cuml import train_test_split
import cudf

X_train, X_test, y_train, y_test = train_test_split(cudf.DataFrame.from_pandas(X), cudf.Series(y), test_size=0.3, random_state=50)


In [8]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
Int64Index: 1184327 entries, 0 to 1186693
Data columns (total 25 columns):
 #   Column                 Non-Null Count    Dtype
---  ------                 --------------    -----
 0   SEXO                   1184327 non-null  uint8
 1   TIPO_PACIENTE          1184327 non-null  uint8
 2   INTUBADO               1184327 non-null  int64
 3   NEUMONIA               1184327 non-null  int64
 4   EDAD                   1184327 non-null  int64
 5   EMBARAZO               1184327 non-null  int64
 6   DIABETES               1184327 non-null  int64
 7   EPOC                   1184327 non-null  int64
 8   ASMA                   1184327 non-null  int64
 9   INMUSUPR               1184327 non-null  int64
 10  HIPERTENSION           1184327 non-null  int64
 11  OTRA_COM               1184327 non-null  int64
 12  CARDIOVASCULAR         1184327 non-null  int64
 13  OBESIDAD               1184327 non-null  int64
 14  RENAL_CRONICA          1184327 non-null  int64
 

In [9]:
from cuml.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf=RandomForestClassifier(random_state=50)
#
X_train=X_train.astype('float32')
y_train=y_train.astype('float32')

clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

classification_report de sklearn

In [10]:
from cuml.ensemble import RandomForestClassifier
from cuml.neighbors import KNeighborsClassifier
import cupy


cuml_RFC = RandomForestClassifier(random_state=50)
cuml_KNC = KNeighborsClassifier()

classifiers = [
    {'Random Forest': cuml_RFC},
    {'KNN': cuml_KNC}
]

for classifier in classifiers:
    name, clf = list(classifier.items())[0]
    clf.fit(X_train.to_cupy(), y_train.to_cupy())
    y_pred = clf.predict(X_test)
    y_pred = cupy.asarray(y_pred)
    print(f'Modelo resultados para {name} usando sklearn')
    print(classification_report(y_test.to_numpy(), y_pred.get()))



Modelo resultados para Random Forest usando sklearn
              precision    recall  f1-score   support

           1       0.97      0.01      0.02      2431
           3       1.00      1.00      1.00    125424
           6       0.97      0.97      0.97     19345
           7       0.99      1.00      0.99    208098

    accuracy                           0.99    355298
   macro avg       0.98      0.75      0.75    355298
weighted avg       0.99      0.99      0.99    355298

Modelo resultados para KNN usando sklearn
              precision    recall  f1-score   support

           1       0.30      0.01      0.03      2431
           3       0.95      0.84      0.89    125424
           6       0.96      0.92      0.94     19345
           7       0.90      0.97      0.93    208098

    accuracy                           0.92    355298
   macro avg       0.78      0.69      0.70    355298
weighted avg       0.91      0.92      0.91    355298



Usando accuracy_score de cuML

In [11]:
# importar accuracy_score
from cuml.metrics import accuracy_score

cuml_RFC=RandomForestClassifier(random_state=50)
cuml_KNC=KNeighborsClassifier()

classifiers=[
    {
        'Ramdom Forest':cuml_RFC
    },
    {
        'KNN':cuml_RFC
    }
]


for classifier in classifiers:
    name, clf = list(classifier.items())[0]
    clf.fit(X_train.to_cupy(), y_train.to_cupy())
    y_pred = clf.predict(X_test)
    y_pred = cupy.asarray(y_pred) # convert to cupy array
    print(f'Modelo resultados para {name} usando cuML')
    print(accuracy_score(y_test.to_numpy(), y_pred.get()))


Modelo resultados para Ramdom Forest usando cuML
0.9917871952056885
Modelo resultados para KNN usando cuML
0.9917871952056885


In [13]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [14]:
! pip install gradio

In [15]:
import gradio as gr
import pandas as pd
import numpy as np


def predict(sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
             diabetes, epoc, asma, inmusupr,
            hipertension, otra_com, cardiovascular, obesidad,
            renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
            resultado_lab, toma_muestra_antigeno, resultado_antigeno,
           uci, dias_hospitalizacion, dias_sintomas):


    # Crea un DataFrame con los valores de entrada
    input_data = pd.DataFrame([[
        sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
       diabetes, epoc, asma, inmusupr,
        hipertension, otra_com, cardiovascular, obesidad,
        renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
        resultado_lab, toma_muestra_antigeno, resultado_antigeno, uci, dias_hospitalizacion, dias_sintomas
        ]], columns=[
        'SEXO', 'TIPO_PACIENTE', 'INTUBADO', 'NEUMONIA', 'EDAD', 'EMBARAZO','DIABETES', 'EPOC',
        'ASMA', 'INMUSUPR','HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD','RENAL_CRONICA',
        'TABAQUISMO', 'OTRO_CASO', 'TOMA_MUESTRA_LAB','RESULTADO_LAB', 'TOMA_MUESTRA_ANTIGENO',
        'RESULTADO_ANTIGENO','UCI', 'DIAS_HOSPITALIZACION', 'DIAS_SINTOMAS'])

    prediction = clf.predict(input_data)
    return prediction




gr.Row
inputs = [
    gr.Radio(choices=[1, 2], label='Sexo'),
    gr.Radio(choices=[1, 2], label='Tipo de Paciente'),
    gr.Radio(choices=[1, 2], label='Intubado'),
    gr.Radio(choices=[1, 2], label='Neumonía'),
    gr.Slider(minimum=0, maximum=120, value=30, label='Edad'),
    gr.Radio(choices=[1, 2], label='Embarazo'),
    gr.Radio(choices=[1, 2], label='Diabetes'),
    gr.Radio(choices=[1, 2], label='EPOC'),
    gr.Radio(choices=[1, 2], label='Asma'),
    gr.Radio(choices=[1, 2], label='Inmunosupresión'),
    gr.Radio(choices=[1, 2], label='Hipertensión'),
    gr.Radio(choices=[1, 2], label='Otra Comorbilidad'),
    gr.Radio(choices=[1, 2], label='Cardiovascular'),
    gr.Radio(choices=[1, 2], label='Obesidad'),
    gr.Radio(choices=[1, 2], label='Enfermedad Renal Crónica'),
    gr.Radio(choices=[1, 2], label='Tabaquismo'),
    gr.Radio(choices=[1, 2], label='Contacto con otro caso'),
    gr.Radio(choices=[1, 2], label='Toma de muestra de laboratorio'),
    gr.Radio(choices=[1, 2], label='Resultado de laboratorio'),
    gr.Radio(choices=[1, 2], label='Toma de muestra de antígeno'),
    gr.Radio(choices=[1, 2], label='Resultado de antígeno'),
    gr.Radio(choices=[1, 2], label='UCI'),
    gr.Number(label='Días de Hospitalización',value=0),
    gr.Number(label='Días con Síntomas',value=0),
]

# Crear componente de salida para Gradio
outputs = gr.Textbox(label="Predicción")

# Crear la interfaz de Gradio
demo = gr.Interface(fn=predict, inputs=inputs, outputs=outputs)

# Ejecutar la aplicación web
if __name__ == "__main__":
    demo.launch(show_api=False,debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
